In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import pandas as pd
import openai
import os
import json
from gpt3_sandbox.api.gpt import GPT
from gpt3_sandbox.api.gpt import Example
from pandasql import sqldf
from tqdm import tqdm
import numpy as np
from AutoReasoner import *

import fasttext.util
# download the fastext embeding model if needed 
# fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('/mnt/idm_automapping/cc.en.300.bin')

In [3]:
def parallel_gpt_reasoner_func(i):
    try:
        prompter = GptReasoner(
            training_df.iloc[i]['id'], 
            training_df.iloc[i]['utterance'], 
            training_df.iloc[i]['context'], 
            training_df.iloc[i]['targetValue'], 
            base_path='./dataset/WikiTableQuestions/',
        )
        prompter.generate_reasonings()
        return {
            'id': training_df.iloc[i]['id'],
            'utterance': training_df.iloc[i]['utterance'],
            'context': training_df.iloc[i]['context'], 
            'targetValue': training_df.iloc[i]['targetValue'],
            'gptReasoning': prompter.gpt_reasoning,
            'gptError': prompter.gpt_error
        }
    except Exception as e:
        return {
            'id': training_df.iloc[i]['id'],
            'uncaught_err': str(e)
        }


maxLimit = float('inf')
training_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/training.tsv', sep='\t', on_bad_lines='warn')
print(training_df.shape)
print(training_df.iloc[0])
n_threads = 10
from joblib import Parallel, delayed
logs = Parallel(n_jobs=n_threads)(delayed(parallel_gpt_reasoner_func)(i) for i in tqdm(range(min(maxLimit, training_df.shape[0]))))
# # logs = Parallel(n_jobs=n_threads)(delayed(parallel_gpt_answer_func)(i) for i in tqdm(range(5)))
# json.dump(logs, open(f'./dataset/WikiTableQuestions/reasonings/GptReasoner_training.json', 'w'))

(14149, 4)
id                                                          nt-0
utterance      what was the last year where this team was a p...
context                                      csv/204-csv/590.csv
targetValue                                                 2004
Name: 0, dtype: object


  0%|▎                                                                                                                                 | 40/14149 [00:05<30:17,  7.76it/s]

KeyboardInterrupt: 

  0%|▎                                                                                                                                 | 40/14149 [00:19<30:17,  7.76it/s]

In [3]:
training_data_autoreasoning = json.load(open(f'./dataset/WikiTableQuestions/reasonings/GptReasoner_training.json', 'r'))
all_demo_embeddings = None
for train_q in tqdm(training_data_autoreasoning):
    if all_demo_embeddings is None:
        all_demo_embeddings = get_utterance_embedding(train_q['utterance'], ft).reshape([-1, 1])
    else:
        all_demo_embeddings = np.append(all_demo_embeddings, get_utterance_embedding(train_q['utterance'], ft).reshape([-1, 1]), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14149/14149 [00:18<00:00, 753.82it/s]


In [5]:
all_demo_embeddings.shape

(300, 14149)

In [ ]:
def parallel_auto_reasoner_func(i):
    try:
        r = GptAutoReasoner(
            test_df.iloc[i]['id'], 
            test_df.iloc[i]['utterance'], 
            test_df.iloc[i]['context'], 
            test_df.iloc[i]['targetValue'] )
        r._gen_NN_demo(training_data_autoreasoning, all_demo_embeddings, ft, demo_num=3)
        r._gen_gpt_prompt()
        r._get_gpt_prediction()
        r._evaluate_result()
        return r._log_dict()
    except Exception as e:
        return {
            'id': test_df.iloc[i]['id'],
            'uncaught_err': str(e)
        }

test_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t', on_bad_lines='skip')
maxLimit = float('inf')
# maxLimit = 20
n_threads = 20
from joblib import Parallel, delayed
res = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_auto_reasoner_func)(i) for i in tqdm(range(min(maxLimit, test_df.shape[0]))))
json.dump(res, open(f'./dataset/WikiTableQuestions/results/AutoReasoner_pristine-unseen-tables_demoNum3.json', 'w'))

 29%|██████████████████████████████████████                                                                                           | 1280/4344 [04:53<12:12,  4.18it/s]

In [15]:
with open(f'./dataset/WikiTableQuestions/results/AutoReasoner_pristine-unseen-tables_demoNum3.json', 'r') as f:
    logs = json.load(f)

    execution_match_cnt = 0
execution_err_cnt = 0
gpt_error = 0

for q in logs:
    
    if 'execution_match' in q and q['execution_match']==True:
        execution_match_cnt += 1
    if 'execution_err' in q and q['execution_err'] is not None:
        execution_err_cnt += 1
    if 'gpt_error' in q and q['gpt_error'] is not None:
        gpt_error += 1
        if gpt_error == 1:
            print(q['gpt_error'])
            
    # if 'target_value' in q and q['target_value'] != q['predicted_value'] and q['target_value'] in q['predicted_value']:
    #     if gpt_error == 0:
    #         print(f"Prediction: {q['predicted_value']}, target: {q['target_value']}")
    #     gpt_error += 1
    #     pass
    
print(f"Execution match: {execution_match_cnt/len(logs):.2f}")
print(f"Execution err: {execution_err_cnt/len(logs):.2f}")
if gpt_error > 0:
    print(f"GPT error: {gpt_error/len(logs):.2f}")    

This model's maximum context length is 8193 tokens, however you requested 12417 tokens (11393 in your prompt; 1024 for the completion). Please reduce your prompt; or completion length.
Execution match: 0.51
Execution err: 0.00
GPT error: 0.09


In [16]:
with open(f'./dataset/WikiTableQuestions/results/AutoReasoner_pristine-unseen-tables_demoNum3.json', 'r') as f:
    logs = json.load(f)
predictions = ''
for q in logs:
    qid = q['id'].replace(' ', '')
    if 'predicted_value' in q:
        result = str(q['predicted_value']).replace('\n', ' ').replace('\t', ' ')
    else:
        result = ''
    predictions += f"{qid}\t{result}\n"
with open(f'./dataset/WikiTableQuestions/results/AutoReasoner_pristine-unseen-tables_demoNum3_predictions.tsv', 'w') as g:
    g.write(predictions)


In [17]:
import subprocess
out = subprocess.check_output(['python2', './dataset/WikiTableQuestions/validate-and-evaluate.py', 
                               '-d', './dataset/WikiTableQuestions/', 
                               '-o', './dataset/WikiTableQuestions/results/output.json',
                               'test', 
                               f'./dataset/WikiTableQuestions/results/AutoReasoner_pristine-unseen-tables_demoNum3_predictions.tsv'])
print(out)

b'Dataset version: "Version 1.0.2 (October 4, 2016)"\nusing test split: pristine-unseen-tables.tsv\nReading split file:  pristine-unseen-tables.tsv\nRead 4344 example IDs\nExample ids:  [\'nu-0\', \'nu-1\', \'nu-2\', \'nu-3\', \'nu-4\', \'nu-5\', \'nu-6\', \'nu-7\', \'nu-8\', \'nu-9\']\nUsing prediction file:  ./dataset/WikiTableQuestions/results/AutoReasoner_pristine-unseen-tables_demoNum3_predictions.tsv\nReading dataset from ./dataset/WikiTableQuestions/tagged/data/training.tagged\nReading dataset from ./dataset/WikiTableQuestions/tagged/data/pristine-seen-tables.tagged\nReading dataset from ./dataset/WikiTableQuestions/tagged/data/pristine-unseen-tables.tagged\nRead 22033 examples\nReading predictions from ./dataset/WikiTableQuestions/results/AutoReasoner_pristine-unseen-tables_demoNum3_predictions.tsv\nExamples: 4344\nCorrect: 2168\nAccuracy: 0.4991\n\n'


In [12]:
print(openai.Completion.create(engine='text-davinci-002',
                                            prompt='What is "goodbye" in Chinese?',
                                ).choices[0].text)



"Goodbye" in Chinese is "再见"
